In [ ]:
import meow as mw
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("default")

## Preparation
let's perform a mode simulation

In [ ]:
length = 10.0
box = mw.Box(
    x_min=-0.22,
    x_max=0.22,
    y_min=0,
    y_max=0.22,
    z_min=0.0,
    z_max=length,
)

In [ ]:
struct = mw.Structure(material=mw.silicon, geometry=box)

In [ ]:
cell = mw.Cell(
    structures=[struct],
    mesh=mw.Mesh2d(
        x=np.linspace(-1, 1, 101),
        y=np.linspace(-1, 1, 101),
    ),
    z_min=0.0,
    z_max=length,
)

In [ ]:
env = mw.Environment(wl=1.55, T=25.0)

In [ ]:
cs = mw.CrossSection.from_cell(
    cell=cell,
    env=env,
)

In [ ]:
mode = mw.compute_modes(cs, 1)[0]

## get the Poynting Vector
$$ \vec{P} = \vec{E} \times \vec{H} $$
calculating the poynting vector is performed under the hood, when one of `Px`,`Py` or `Pz` is requested


In [ ]:
mode.Px.shape

It can be used with the native visualization functions

In [ ]:
mw.visualize(mode, fields=["Ex"])
mw.visualize(mode, fields=["Pz"])

## Mode area calculation

$$ A_{eff} = \frac{\left(\int|\vec{E}|^2dA\right)^2}{\int|\vec{E}|^4dA} $$

In [ ]:
mode.A

## Compare to integrate interpolate

We can also itegrate over the modal fields and derived quantities by first finding a suitable interpolation function. This is computationally much more expensive and not necessarily more accurate. It however provides tha benefit, that the integration can be performed over a region of interest, not limited to integer mesh positions. 

For the calculation of the effective mode area this is however not beneficial. Just as a demonstration:

In [ ]:
from scipy.linalg import norm
from meow.integrate import integrate_interpolate_2d as integrate_2d
import numpy as np

In [ ]:
vecE = np.stack([mode.Ex, mode.Ey, mode.Ez], axis=-1)
E_sq = norm(vecE, axis=-1, ord=2)
E_qu = E_sq**2
x = mode.cs.mesh.x_
y = mode.cs.mesh.y_
integrate_2d(x, y, E_sq) ** 2 / integrate_2d(x, y, E_qu)